# PyTorch MNIST Example

Набор данных MNIST представляет собой большую базу рукописных цифр, которая обычно используется для обучения различных систем обработки изображений. База данных также широко используется для обучения и тестирования в области машинного обучения.

Он состоит из 60000 изображений для трейна и 10000 тестовых изображений.

![MNIST Examples](mnist.png)

## Import all the necesary libraries

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim

from torch.utils.data import TensorDataset
from torchvision import transforms
from torchvision.datasets import MNIST

import matplotlib.pyplot as plt
import time

In [ ]:
# если гпу у вас несколько укажите номер гпу, которую хотите использовать
# номер свободной можно посмотреть с помощью nvidia-smi в терминале
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

## Download the MNIST Train and Test set 
Используя torch.utils.data DataLoader, мы перемешиваем данные и устанавливаем размера батча равный 256

In [ ]:
train = MNIST('./data', train=True, download=True, transform=transforms.ToTensor())

test = MNIST('./data', train=False, download=True, transform=transforms.ToTensor())

cuda = torch.cuda.is_available()

dataloader_args = dict(shuffle=True, batch_size=256,num_workers=4, pin_memory=True) if cuda \
                        else dict(shuffle=True, batch_size=64)
train_loader = dataloader.DataLoader(train, **dataloader_args) 
test_loader = dataloader.DataLoader(test, **dataloader_args)

## Compute basic data statistics
Всегда важно посчитать размер данных и основные статистики min/max и mean/variance

In [ ]:
train_data = train.train_data
train_data = train.transform(train_data.numpy())

print('[Train]')
print(' - Numpy Shape:', train.train_data.cpu().numpy().shape)
print(' - Tensor Shape:', train.train_data.size())
print(' - min:', torch.min(train_data))
print(' - max:', torch.max(train_data))
print(' - mean:', torch.mean(train_data))
print(' - std:', torch.std(train_data))
print(' - var:', torch.var(train_data))

## Visualize a few training samples
Используем matplotlib.pyplot для визуализации данных

In [ ]:
# Visualize a training instance with matplotlib
plt.imshow(train.train_data.cpu().numpy()[0], cmap='gray')

In [ ]:
plt.imshow(train.train_data.cpu().numpy()[1], cmap='gray')

## Define our Neural Network Model 
Определим нашу модель используя класс torch.nn.Module

In [ ]:
# SIMPLE MODEL DEFINITION
class Simple_MLP(nn.Module):
    def __init__(self, size_list):
        super(Simple_MLP, self).__init__()
        layers = []
        self.size_list = size_list
        for i in range(len(size_list) - 2):
            layers.append(nn.Linear(size_list[i],size_list[i+1]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(size_list[-2], size_list[-1]))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(-1, self.size_list[0]) # Flatten the input
        return self.net(x)

## Create the model and define the Loss and Optimizer

Создаем инстанс модели Simple_MLP передавая ей размеры слоев в качестве праметров. Входной слой содержит 28*28 = 784 нейронов. Мы определяем скрытый слой размера 256, и выходной слой размера 10, соответствующий  вероятностям 10 цифр (0-9).

Так как это задача классификации, будем использовать Cross Entropy Loss. Мы определяем лосс (критерий) как torch.nn.CrossEntropyLoss.

Для обучения сети воспользуемся torch.optim.SGD optimizer.

In [ ]:
model = Simple_MLP([784, 256, 10])
criterion = nn.CrossEntropyLoss()
print(model)

optimizer = optim.Adam(model.parameters())
device = torch.device("cuda" if cuda else "cpu")

## Create a function that will train the network for one epoch

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    model.to(device)

    running_loss = 0.0
    
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   
        data = data.to(device)
        target = target.long().to(device)

        outputs = model(data)
        loss = criterion(outputs, target)
        running_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    end_time = time.time()
    
    running_loss /= len(train_loader)
    print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')
    return running_loss

## Create a function that will evaluate our network's performance on the test set

In [ ]:
def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()
        model.to(device)

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0

        for batch_idx, (data, target) in enumerate(test_loader):   
            data = data.to(device)
            target = target.long().to(device)

            outputs = model(data)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()

            loss = criterion(outputs, target).detach()
            running_loss += loss.item()


        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        print('Testing Loss: ', running_loss)
        print('Testing Accuracy: ', acc, '%')
        return running_loss, acc


## Train the model for N epochs
Мы тренируем и тестируем сеть в цикле, отслеживая лоссы и точность (accuracy)

In [ ]:
n_epochs = 10
Train_loss = []
Test_loss = []
Test_acc = []

for i in range(n_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    test_loss, test_acc = test_model(model, test_loader, criterion)
    Train_loss.append(train_loss)
    Test_loss.append(test_loss)
    Test_acc.append(test_acc)
    print('='*20)

## Visualize Training Data

In [ ]:
plt.title('Training Loss')
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.plot(Train_loss)

In [ ]:
plt.title('Test Loss')
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.plot(Test_loss)

In [ ]:
plt.title('Test Accuracy')
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy (%)')
plt.plot(Test_acc)